In [32]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance

# json.dumps() 사용하기 위해 필요
import json


# Tool Implement
def get_ticker(inputs):  # Parameters object를 받음
    ddg = DuckDuckGoSearchAPIWrapper(backend="api")
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


# 손익계산서
def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)

    # LLM은 String을 읽기 때문에 String 형태로 변환해야함
    # stock은 데이터프레임(스프레드시트) 형태로 들어옴
    # .to_json(): 데이터프레임 형태를 json 형태로 변환
    # json.dumps(): json 형태를 다시 문자열로 변환
    return json.dumps(stock.income_stmt.to_json())  # stmt: statement


# 대차대조표
def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


# 100일 동안의 주식 퍼포먼스
def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    # history에 들어가보면 바꿀 수 있는 parameter를 확인할 수 있음
    return json.dumps(stock.history(period="3mo").to_json())


functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}

# Tools에 넘겨줄 Description
functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [9]:
from openai import OpenAI

client = OpenAI()

# # OpenAI에 assistant를 생성함
# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4o-mini",
#     # tools가 어떻게 작동될 지에 대한 description을 List형태로 넘겨줘야 함
#     tools=functions,
# )

# print(assistant)
# # assistant.id를 확인할 수 있는데 이것은 앞으로도 계속 사용됨
# # assistant를 생성한 후에 OpenAI의 Playground를 이용해 assistant를 테스트해 볼 수 있음

assistant_id = "asst_sgTX3TAy06FPU13MqF1zOeqw"

In [43]:
# thread 생성
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Cloudflare stock is a good buy",
        }
    ]
)
thread
# Assistant의 thread에 가보면 생성된 것을 확인할 수 있음

Thread(id='thread_F7Oz93ouBEOnDc74dC94A3yM', created_at=1737793823, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [44]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_tzkQMC603pjt3AwFloH5eYFu', assistant_id='asst_sgTX3TAy06FPU13MqF1zOeqw', cancelled_at=None, completed_at=None, created_at=1737793842, expires_at=1737794442, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_F7Oz93ouBEOnDc74dC94A3yM', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=FunctionDefini

In [23]:
# Thread 실행
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


# Thread에 메시지 보내기
def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content,
    )


# Thread에 있는 메시지 받기
def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


# 어떤 Tool을 사용해야 하는지 받아오고 이를 실행시킴
def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            # outputs 리스트에 dictionary로 넣어줌
            {
                # functions_map을 이용하여 string으로 받아온 것을 바로 함수로 연결시켜줄 수 있음
                # json.loads를 이용하여 json 객체로 만들어 줌
                # "get_ticker"
                # -> get_ticker({"company_name":"Salesforce"})
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outpus,
    )

In [57]:
get_run(run.id, thread.id).status
# 'completed': AI가 응답을 완료했음을 나타냄
# 'required_action': 추가적인 조치를 하여야 함을 나타냄

'completed'

In [58]:
get_messages(thread.id)

user: I want to know if the Cloudflare stock is a good buy
assistant: To determine whether Cloudflare (Ticker: NET) is a good buy, let's analyze key financial information and recent stock performance.

### 1. Financial Overview

**Income Statement Highlights:**
- **Total Revenue:** $1.296 billion
- **Gross Profit:** $989.7 million
- **Net Loss:** $183.9 million
- **Diluted EPS:** -$0.55
- **Total Operating Expenses:** $1.482 billion
- **Gross Profit Margin:** Approximately 76.4%

**Balance Sheet Highlights:**
- **Total Assets:** $2.759 billion
- **Total Liabilities:** $1.997 billion
- **Equity:** $763 million
- **Total Debt:** $1.435 billion
- **Working Capital:** $1.416 billion

### 2. Recent Stock Performance
- **Current Price:** $117.16
- **52-Week Range:** This indicates the volatility and price movement. The current price is higher within its operating range, which typically indicates upward momentum.
- **Performance Trend:** Over the past 100 days, the stock has shown fluctuation

In [ ]:
send_message(thread.id, "Please go ahead!")

In [50]:
get_tool_outputs(run.id, thread.id)

Calling function: get_income_statement with arg {"ticker": "NET"}
Calling function: get_balance_sheet with arg {"ticker": "NET"}
Calling function: get_daily_stock_performance with arg {"ticker": "NET"}


[{'output': '"{\\"1703980800000\\":{\\"Tax Effect Of Unusual Items\\":-10563000.0,\\"Tax Rate For Calcs\\":0.21,\\"Normalized EBITDA\\":14130000.0,\\"Total Unusual Items\\":-50300000.0,\\"Total Unusual Items Excluding Goodwill\\":-50300000.0,\\"Net Income From Continuing Operation Net Minority Interest\\":-183949000.0,\\"Reconciled Depreciation\\":135820000.0,\\"Reconciled Cost Of Revenue\\":307005000.0,\\"EBITDA\\":-36170000.0,\\"EBIT\\":-171990000.0,\\"Net Interest Income\\":62295000.0,\\"Interest Expense\\":5872000.0,\\"Interest Income\\":68167000.0,\\"Normalized Income\\":-144212000.0,\\"Net Income From Continuing And Discontinued Operation\\":-183949000.0,\\"Total Expenses\\":1482230000.0,\\"Total Operating Income As Reported\\":-185485000.0,\\"Diluted Average Shares\\":333656000.0,\\"Basic Average Shares\\":333656000.0,\\"Diluted EPS\\":-0.55,\\"Basic EPS\\":-0.55,\\"Diluted NI Availto Com Stockholders\\":-183949000.0,\\"Net Income Common Stockholders\\":-183949000.0,\\"Net Incom

In [53]:
submit_tool_outputs(run.id, thread.id)

Calling function: get_income_statement with arg {"ticker": "NET"}
Calling function: get_balance_sheet with arg {"ticker": "NET"}
Calling function: get_daily_stock_performance with arg {"ticker": "NET"}


Run(id='run_tzkQMC603pjt3AwFloH5eYFu', assistant_id='asst_sgTX3TAy06FPU13MqF1zOeqw', cancelled_at=None, completed_at=None, created_at=1737793842, expires_at=1737794442, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1737793914, status='queued', thread_id='thread_F7Oz93ouBEOnDc74dC94A3yM', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=Function